In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import pandas as pd
import sys
import json
#%pip install --disable-pip-version-check pycaret
sys.path.append("../src")
from funcoes import *
import seaborn as sns
from funcoes_ import perfil_base_conversao,plot_safra_conversion_rate,detectar_tipos,target_balance,resumo_missing,resumo_numericas,correlacao_numericas,chi2_categoricas,woe_iv_todas,matriz_correlacao,build_missing_plan,fit_missing_prep,apply_missing_prep,aplicar_dtypes,eliminacao_progressiva_por_importancia,grid_search_inteligente_lgbm_gini,_detectar_categoricas,_padronizar_categorias_para_fit,_alinhar_colunas_para_predict,_aplicar_categorias_salvas,avaliar_modelo_roc,plot_ks_prob_side_by_side,avaliar_por_limiar,plot_confusion_from_metrics,treinar_e_salvar_modelo_lgbm,aplicar_modelo_pkl,plot_ks_prob_side_by_side
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

###1- Train & Test

####1.1 - General Checks

In [0]:
#Após recebermos uma base enriquecida precisamos dividir entre Treino e Teste, para evitar Data Leakage.
#Data Leakage é uma falha que acontece durante os testes de um modelo de machine learning,
#no qual informações são compartilhadas entre um conjunto de dados usado no treinamento
#e outro conjunto de dados que é usado para validar essa modelagem, também chamado de dataset de testes.
#

In [0]:
base_modelagem = pd.read_csv("../data/processed/base_modelagem.csv", sep=",")

In [0]:
base_modelagem.head()

In [0]:
#A função perfil_base auxilia na verificação de métricas básicas da base de dados
resultado = perfil_base_conversao(base_modelagem, id_col='ID', target_col='target_sucesso', safra_col='time_since_test_start')

In [0]:
base_modelagem["time_since_test_start"] = base_modelagem["time_since_test_start"].astype(float).astype("int")
safra_br = plot_safra_conversion_rate(base_modelagem, safra_col="time_since_test_start", conversao_col="target_sucesso", conv_rate_min=0, conv_rate_max=0.8)

In [0]:
safra_br

In [0]:
#verificando a tipagem das variáveis
base_modelagem.dtypes.to_frame().value_counts(0)

In [0]:
#verificando a quantidade de safras e ofertas
base_modelagem[['time_since_test_start','offer_id']].value_counts().to_frame().reset_index().sort_values('time_since_test_start')


####1.2 - Train Test Division

In [0]:
#Apesar da visão de "safras" vamos resolver esse problema apenas comm Treino e Teste OOS
#Isso porque estamos falando de apenas um mes de campanha com novos disparos consecutivos
#As variáveis nao devem perder poder preditivo em um período de tempo tão curto.

In [0]:
base_modelagem.shape

In [0]:
#A base de treino deve ser dividida em Desenvolvimento e validação (train test)
#Para que possamos avaliar os modelos dentro do mesmo contexto temporal do treinamento OOS
# A base -> treino precisa ser dividida em train, test oos (testar a performance nas safras conhecidas)
# 30% da base é o teste oos.
train, test_oos = train_test_split(base_modelagem, test_size=0.3, stratify=base_modelagem['target_sucesso'], random_state=42)
train.shape, test_oos.shape

In [0]:
#verificando as taxas de conversao
train['target_sucesso'].value_counts()/len(train)*100

In [0]:
test_oos['target_sucesso'].value_counts()/len(test_oos)*100

In [0]:
#Salvando as bases. A partir daqui todas as análises e transformacoes serao realizadas no conjunto de treino
#E aplicada quando necessário no conjunto de testes
train.to_csv("../data/processed/train.csv",sep=",",index=False,header=True)
test_oos.to_csv("../data/processed/test_oos.csv",sep=",",index=False,header=True)

### 2 - Exploratory Data Analysis (EDA)

In [0]:
train= pd.read_csv("../data/processed/train.csv",sep=",")

In [0]:
train.head()

In [0]:
#Analisar os dados!! Quais variáveis vamos manter? descartar? transformar ou criar? Quais os Insights?
target = "target_sucesso"

####2.1 - Types

In [0]:
target = "target_sucesso"

num_cols, cat_cols = detectar_tipos(
    train,
    target=target,
    exclude_cols=["ID","cliente_id","offer_id","time_since_test_start"],  # não classificar esses
    force_cat=["genero","offer_type","safra_registro","recebeu_email","recebeu_mobile","recebeu_social","recebeu_web",
        "social","web","mobile","email"],                   # garantir que fiquem categóricas
    force_num=[
        "canais_recebidos_total","discount_value","duration","min_value",
        "n_instancias_anteriores_mesma_oferta","qtd_ofertas_completas_validas",
        
    ],
    int_low_card_as_cat=False  # não transformar ints de baixa cardinalidade em categorias
)

print("Categóricas:", cat_cols)
print("Numéricas (amostra):", num_cols[:10], " ... total:", len(num_cols))


In [0]:
num_cols

In [0]:
cat_cols

In [0]:
for col in cat_cols:
    if col in train.columns:
        train[col] = train[col].astype('category')

In [0]:
train.dtypes

In [0]:
tipagem = train.dtypes.reset_index()
dicionario_tipagem = dict(zip(tipagem["index"], tipagem[0]))

In [0]:
dicionario_tipagem

####2.2 - Target

In [0]:
target_balance(train, target)

####2.3 - Missing

In [0]:
#avaliacao rapida de missings na base, posteriormente vamos filtrar os campos com mais de 40% de missing
drop_missing = ["dias_desde_ultima_conversao_valida_mesma",
"dias_desde_ultima_oferta_mesma",
"n_conversoes_validas_anteriores_mesma",
"tempo_medio_conversao",
"dias_desde_ultima_conversao_valida",
"tempo_medio_visualizacao"]

resumo_missing(train).head(20)

####2.4 - Variables

In [0]:
num_cols

In [0]:
#Essas variáveis possuem problemas de inconsistencia devemos rever o conceito no book
drop_inconsistencia = ["mobile_ratio_historico","social_ratio_historico","dias_desde_ultima_conversao_valida_mesma","mobile_ratio_historico","social_ratio_historico","tempo_medio_conversao","web_ratio_historico","dias_desde_ultima_transacao.1"]

resumo_numericas(train, num_cols)

In [0]:
#p_valor baixo, vamos manter todas
chi2_categoricas(train, cat_cols, target)

In [0]:
# 5) Correlação + pares fortes
corr, pares_fortes = correlacao_numericas(train, num_cols)
display(pares_fortes)

In [0]:
#remover variaveis com alto colinearidade
drop_alta_colinearidade = pares_fortes["var2"].tolist()

In [0]:
drop_alta_colinearidade

In [0]:
# 7) WoE/IV (ranking)
#IV/WOE → mede capacidade de discriminação da variável para um problema binário.
#A variável ["n_conversoes_validas_anteriores_mesma"] tem um IV absurdo provavelmente algum problema na construcao da variavel (leakeage)
#Devemos remover ou consertar o conceito
#Vale lembrar que estamos falando de correlacoes lineares, portanto nao vamos nos livrar 
#de variaveis que apenas tem iv baixo.
drop_iv_absurdo = ["n_conversoes_validas_anteriores_mesma"]
display(woe_iv_todas(train, num_cols + cat_cols, target, n_bins=10, metodo="quantile"))

In [0]:
drop_missing

In [0]:
remover_vars = list(set(drop_inconsistencia+drop_missing+drop_alta_colinearidade+drop_iv_absurdo))

In [0]:
resumo_missing(train)

In [0]:
train.columns

In [0]:
train.dtypes

In [0]:
remover_vars

In [0]:
vars_sobreviventes = [x for x in train if x not in remover_vars]

In [0]:
vars_sobreviventes

In [0]:
#Sobraram 49 variáveis!!
len(vars_sobreviventes)

In [0]:
train_selecionado = train[vars_sobreviventes]

In [0]:
ids_ =["ID","time_since_test_start","offer_id","cliente_id"]
target="target_sucesso"

In [0]:
#correlacao maxima de 0.8
matriz_correlacao(train_selecionado.drop(ids_,axis=1).iloc[:, 1:15])

###3 - Data Prep

In [0]:
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo

In [0]:
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo

####3.1 - Handling Missing Values

In [0]:
# Nesta etapa importante vamos verificar nossas variáveis e propor um tratamento adequado para os valores faltantes:

In [0]:
train_selecionado.head()

In [0]:
train_selecionado.dtypes

In [0]:
missing_plan_df = build_missing_plan()
missing_plan_df

In [0]:
train_selecionado

In [0]:
#A variável idade é um tanto complicada, nao sabemos o real motivo dela estar faltando mas vamos 
#partir da premissa que se trata de um missing aleatório.
#Para tentar capturar quando o missing ocorreu vamos criar uma flag_missing, para que o modelo tenha chance de
#aprender quando o missing ocorreu.
train_selecionado["idade_missing"] = train_selecionado["idade"].isna().astype(int)

In [0]:
train_selecionado["idade"].count()

In [0]:
train_selecionado["idade_missing"].value_counts()

In [0]:
#Definindo o plano de imputação
plan = build_missing_plan()

prep = fit_missing_prep(train_selecionado, plan)

#salvar e dar load no prep
with open("../model_artefacts/missing_plan.json", "w", encoding="utf-8") as f:
    json.dump(prep, f, ensure_ascii=False, indent=4)

#ler arquivo
with open("../model_artefacts/missing_plan.json", "r", encoding="utf-8") as f:
    prep = json.load(f)


train_tratado = apply_missing_prep(train_selecionado, prep, copy=True)

# Para o teste:
# test_tratado = apply_missing_prep(test, prep, copy=True)


In [0]:
train_tratado.head()

In [0]:
#As variáveis de oferta permaneceram, muito bom para a nossa estratégia de escoragem
#variar as ofertas para o cliente e verificar maior probabilidade de conversao!
variaveis_de_oferta = ["offer_id","offer_type","discount_value","min_value","duration","web","email","mobile","social"]
resumo_missing(train_tratado).head(20)

In [0]:
# Dicionário de tipos
dtypes_dict = {
    "ID": "object",
    "target_sucesso": "int64",
    "cliente_id": "object",
    "offer_id": "object",
    "time_since_test_start": "int64",
    "idade": "float64",
    "genero": "category",
    "limite_credito": "float64",
    "safra_registro": "category",
    "historico_conversao": "float64",
    "dias_desde_ultima_transacao": "float64",
    "media_tempo_entre_transacoes": "float64",
    "transacoes_distintas_dias": "float64",
    "qtd_ofertas_anteriores": "float64",
    "qtd_ofertas_visualizadas": "float64",
    "qtd_ofertas_completas_validas": "int64",
    "taxa_visualizacao": "float64",
    "taxa_conversao": "float64",
    "offer_type": "category",
    "discount_value": "int64",
    "min_value": "int64",
    "duration": "int64",
    "web": "category",
    "email": "category",
    "mobile": "category",
    "social": "category",
    "dias_desde_ultima_oferta_recebida": "float64",
    "dias_desde_ultima_oferta_visualizada": "float64",
    "qtd_transacoes_7d": "float64",
    "qtd_transacoes_14d": "float64",
    "qtd_ofertas_recebidas_14d": "float64",
    "qtd_ofertas_visualizadas_14d": "float64",
    "qtd_conversoes_validas_14d": "float64",
    "canais_recebidos_total": "int64",
    "recebeu_web": "category",
    "recebeu_email": "category",
    "recebeu_mobile": "category",
    "recebeu_social": "category",
    "canais_preferidos": "float64",
    "email_ratio_historico": "float64",
    "taxa_visualizacao_historica_tipo": "float64",
    "taxa_conversao_historica_tipo": "float64",
    "taxa_visualizacao_historica_oferta": "float64",
    "taxa_conversao_historica_canal_social": "float64",
    "afinidade_canais_conv": "float64",
    "taxa_conversao_historica_bucket_duracao": "float64",
    "taxa_conversao_historica_bucket_desconto": "float64",
    "taxa_conversao_historica_bucket_minimo": "float64",
    "n_instancias_anteriores_mesma_oferta": "int64",
    "idade_missing": "int64"
}


In [0]:
#Salvando o dicionario de tipos para usarmos posteriormente
import json
# Salvar
with open("../model_artefacts/train_dtypes.json", "w", encoding="utf-8") as f:
    json.dump(dtypes_dict, f, ensure_ascii=False, indent=4)

In [0]:
# Carregar dicionario de tipos
with open("../model_artefacts/train_dtypes.json", "r", encoding="utf-8") as f:
    dtypes_dict = json.load(f)

df = aplicar_dtypes(train_tratado, dtypes_dict)
print(df.dtypes)

####3.2 - Other treatments

**Normalização**:<br>
Alguns modelos são sensíveis a magnitude, das variáveis, <br>
mas não é o caso do modelo de lightGBM que vamos treinar.<br>
Deixar de normalizar mantém as variáveis, sendo mais fácil de interpretar.<br>

**Outliers**:<br>
O modelo também nao é sensível a outliers.<br>
Os outliers proveniente de erros, foram observados em etapoas anteriores.<br>

**One Hot Enconding**:<br>
Sem necessidade, cria um monte de coluna desnecessária.<br>
Nosso modelo agrupa estatisticamente as categorias (ordena pelo target e busca splits).<br> 

Agora podemos seguir em paz

In [0]:
#Salvando base treino_tratada
#train_tratado.to_csv("../data/processed/train_tratado.csv", sep=",",header=True,index=False)

###4 - Variable Selection

Na verdade estamos selecionando variáveis desde nosso passo de EDA.<br>
Mas aqui o foco é de fato nas variáveis que podem nos ajudar a prever o comportamento do cliente ao receber uma oferta.

####4.1 - Recursive Feature Elimination (RFE)

In [0]:

#Carregando base treino_tratada
#Inferindo o schema
train_tratado = pd.read_csv("../data/processed/train_tratado.csv", sep=",")
train_tratado = aplicar_dtypes(train_tratado, dtypes_dict)

In [0]:
train_tratado.dtypes

In [0]:
colunas_id = ['ID', 'cliente_id', 'offer_id', 'time_since_test_start']
target = 'target_sucesso'

resultados_df, figs = eliminacao_progressiva_por_importancia(
    df, colunas_id, target,
    min_features=5,   # pare quando restarem 5 variáveis
    max_steps=None,   # ou limite por número de etapas, ex: 20
    plot=True
)


In [0]:
#A Etapa número 9 é uma boa opçao para seguirmos,
#Mantém a performance e diminui o tempo de treinamento
pd.set_option('display.max_colwidth', 50)
resultados_df

In [0]:
figs['gini']

In [0]:
figs['ks']

###5 - Model Train

####5.1 - Grid Search

In [0]:
#Agora que já tratamos o dado e selecionamos as variáveis, vamos fazer uma modelagem mais robusta,
#GridSearch para procurar os melhores hiperparâmetros para nosso modelo
#Vamos procurar recursivamente para diversos experimentos possíveis:
param_grid = {
        # taxa de aprendizado: menor -> mais árvores, maior estabilidade; maior -> converge rápido, risco overfit
        'learning_rate': [0.01, 0.05, 0.1],
        # controle do tamanho das folhas: mais folhas capturam interações complexas, mas podem overfit
        'num_leaves': [30, 63, 127],
        # profundidade máxima: -1 = ilimitado; valores moderados ajudam a regularizar
        'max_depth': [3, 6, 8, 10],
        # amostra mínima por folha: maior -> mais suave/regularizado
        'min_child_samples': [20, 50, 100],
        # amostragem de linhas por árvore (bagging): <1 ajuda a reduzir variância
        'subsample': [0.7, 0.9, 1.0],
        # amostragem de colunas por árvore: <1 ajuda a reduzir correlação entre árvores
        'colsample_bytree': [0.7, 0.9, 1.0],
        # L1 e L2, penalizações que ajudam a controlar complexidade
        'reg_alpha': [0.0, 0.1, 0.5],
        'reg_lambda': [0.0, 0.1, 0.5],
        # ganho mínimo para dividir (split) — aumenta a exigência para criar novas folhas
        'min_split_gain': [0.0, 0.1]
    }

In [0]:
#Vamos carregar nossa base de dados novamente, para nao dependermos de rodar o notebook todo.
# Carregar dicionario de tipos
with open("../model_artefacts/train_dtypes.json", "r", encoding="utf-8") as f:
    dtypes_dict = json.load(f)

train_tratado = pd.read_csv("../data/processed/train_tratado.csv", sep=",")
train_tratado = aplicar_dtypes(train_tratado, dtypes_dict)

In [0]:
explicativas_selecionadas = resultados_df.iloc[9].tolist()[4]
explicativas_selecionadas

In [0]:
explicativas_selecionadas = ['idade',
 'genero',
 'limite_credito',
 'safra_registro',
 'historico_conversao',
 'dias_desde_ultima_transacao',
 'media_tempo_entre_transacoes',
 'transacoes_distintas_dias',
 'qtd_ofertas_anteriores',
 'qtd_ofertas_visualizadas',
 'qtd_ofertas_completas_validas',
 'taxa_visualizacao',
 'taxa_conversao',
 'offer_type',
 'discount_value',
 'min_value',
 'duration',
 'social',
 'dias_desde_ultima_oferta_recebida',
 'dias_desde_ultima_oferta_visualizada',
 'qtd_transacoes_7d',
 'qtd_transacoes_14d',
 'qtd_ofertas_recebidas_14d',
 'qtd_ofertas_visualizadas_14d',
 'qtd_conversoes_validas_14d',
 'canais_recebidos_total',
 'email_ratio_historico',
 'taxa_visualizacao_historica_tipo',
 'taxa_conversao_historica_tipo',
 'taxa_visualizacao_historica_oferta',
 'taxa_conversao_historica_canal_social',
 'afinidade_canais_conv',
 'taxa_conversao_historica_bucket_duracao',
 'taxa_conversao_historica_bucket_desconto',
 'taxa_conversao_historica_bucket_minimo',
 'n_instancias_anteriores_mesma_oferta']

In [0]:
train_tratado = train_tratado[['ID', 'offer_id', 'cliente_id','time_since_test_start']+[target]+explicativas_selecionadas]

In [0]:
train_tratado.head()

In [0]:
variaveis_de_oferta = ["offer_id","offer_type","discount_value","min_value","duration","web","email","mobile","social"]

In [0]:
#verificando se todas as variáveis de ofertas sobreviveram
[x for x in variaveis_de_oferta if x not in train_tratado.columns]

In [0]:
best_params, historico = grid_search_inteligente_lgbm_gini(
    df=train_tratado.drop(["ID","offer_id","cliente_id","time_since_test_start"], axis=1),
    target='target_sucesso',   # troque pelo nome do seu alvo
    categorical_cols=None,     # ou passe a lista, ex.: ['genero','offer_type',...]
    cv_splits=5,
    early_stopping_rounds=100,
    max_combinations=60,
    random_state=42,
    verbose=1
)


In [0]:
print(historico.head(10))
print("\nBest params:\n", best_params)

In [0]:
parametros_escolhidos = {'objective': 'binary', 'boosting_type': 'gbdt', 'random_state': 42, 'n_jobs': -1, 'learning_rate': 0.01, 'num_leaves': 63, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 1.0, 'colsample_bytree': 0.9, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'min_split_gain': 0.0, 'n_estimators': 1340}

In [0]:
parametros_escolhidos

####5.2 - Final Model

In [0]:
id_cols = ["ID","offer_id","cliente_id","time_since_test_start"]

importance_df, train_escorado, modelo_pkl = treinar_e_salvar_modelo_lgbm(
    parametros_escolhidos=parametros_escolhidos,
    train_tratado=train_tratado,     # base ORIGINAL sem dropar
    target='target_sucesso',
    path='../model_artefacts/',
    score_col='score_modelo',
    id_cols=id_cols                  # <--- agora aqui!
)

In [0]:
train_escorado.head()

In [0]:
importance_df.head()

In [0]:
train_escorado.head()

In [0]:
importance_df

###6 - Model Testing

In [0]:
#Agora que já treinamos nosso modelo vamos escorar no dataset de Testes
#Precisamos aplicar todas as transformaçoes feitas no treino

In [0]:
#Ler base de testes
teste_oos = pd.read_csv("../data/processed/test_oos.csv", sep=",")
teste_oos.shape

#Adicionar a coluna idade_missing
teste_oos["idade_missing"] = teste_oos["idade"].isna().astype(int)

#resgatando colunas necessarias
colunas = pd.read_csv("../data/processed/train_tratado.csv", sep=",", nrows=0).columns
teste_oos = teste_oos[colunas]

#Resgatando os dtypes e Tipando a base de testes
with open("../model_artefacts/train_dtypes.json", "r", encoding="utf-8") as f:
    dtypes_dict = json.load(f)

teste_oos_tratado = aplicar_dtypes(teste_oos, dtypes_dict)

#Imputar os Missings
#ler arquivo
with open("../model_artefacts/missing_plan.json", "r", encoding="utf-8") as f:
    prep = json.load(f)

teste_oos_tratado = apply_missing_prep(teste_oos_tratado, prep, copy=True)


In [0]:
#Agora podemos escorar o conjunto de testes
teste_escorado = aplicar_modelo_pkl(
    df_teste=teste_oos_tratado,              # df apenas com explicativas
    path='../model_artefacts/',
    score_col='score_modelo'
)

In [0]:
teste_escorado.head()

In [0]:
teste_escorado.to_csv("../model_artefacts/teste_escorado.csv",sep=",",index=False)

###7 - Model Evaluation

Vamos avaliar o modelo com algumas métricas para verificar se ele está performando bem:<br>

**Acurácia**: “% que acertei no geral”<br>
**Precisão**: “Quando digo que é positivo, acerto quantas vezes?”<br>
**Recall**: “Entre todos os positivos, quantos consegui pegar?”<br>
**F1**: “Equilíbrio entre precisão e recall”<br>
**AUC / Gini**: “Quão bem separo as classes no geral”<br>
**KS**:“Distância máxima entre as curvas de acerto e erro”

In [0]:
train_escorado.head()

In [0]:
#Ler as bases escoradas para iniciarmos a avaliaçao
train_escorado = pd.read_csv("../model_artefacts/train_escorado.csv",sep=",")
teste_escorado = pd.read_csv("../model_artefacts/teste_escorado.csv",sep=",")
ids_ = ["ID","cliente_id","offer_id","time_since_test_start"]
target = "target_sucesso"

In [0]:
metrics = avaliar_modelo_roc(
    df_train=train_escorado,  # df com colunas score & target
    df_test=teste_escorado,
    score_col="score_modelo",
    target_col="target_sucesso",
    title="ROC - LGBM"
)

In [0]:
info = plot_ks_prob_side_by_side(
    df_train=train_escorado,
    df_test=teste_escorado,
    score_col="score_modelo",
    target_col="target_sucesso",
    title="KS por Probabilidade — LGBM"
)

In [0]:
# o KS é máximo quando a prob de conversao é 0.44
#Entao vou utilizar esse corte para avaliar
metricas = avaliar_por_limiar(
    df_train=train_escorado,
    df_test=teste_escorado,
    score_col="score_modelo",
    target_col="target_sucesso",
    limiar=0.44
)


In [0]:
#Quando o modelo diz que é uma conversao, ele acerta 78%.
#Entre todas as conversoes quantas o modelo acertou? 85%
metricas

In [0]:
plot_confusion_from_metrics(metricas)

In [0]:
#O modelo generaliza bem, pois as métricas de treino e teste são próximas.
#Alta AUC e KS indicam ótima separação entre positivos e negativos.
#O recall alto é importante para capturar o maior número de conversoes.
#Precisão também está boa, evitando muitas predições erradas positivas.
#Overfitting praticamente inexistente pois a performance caiu pouco de treino para teste.
#Pode ser usado em produção com segurança.

###8 - Model Inference

####8.1 - Offer Recomendation

Vamos transformar o modelo (treinado na granularidade cliente‑oferta) em um motor de recomendação que,<br>
na hora da escoragem, testa todas as opções de oferta para cada cliente e devolve uma tabela de **melhor oferta por cliente**<br>

In [0]:
#Essas sao as informações que variam conforme a ofera
variaveis_de_oferta = ["offer_id","offer_type","discount_value","min_value","duration","web","email","mobile","social"]

In [0]:
offers = pd.read_csv("../data/processed/processed_offers.csv",sep=',') 

In [0]:
#Essas sao as ofertas disponíveis
offers

In [0]:
#Tratando a base de testes

In [0]:
#Ler base de testes
teste_oos = pd.read_csv("../data/processed/test_oos.csv", sep=",")
teste_oos.shape

#Adicionar a coluna idade_missing
teste_oos["idade_missing"] = teste_oos["idade"].isna().astype(int)

#resgatando colunas necessarias
colunas = pd.read_csv("../data/processed/train_tratado.csv", sep=",", nrows=0).columns
teste_oos = teste_oos[colunas]

#Resgatando os dtypes e Tipando a base de testes
with open("../model_artefacts/train_dtypes.json", "r", encoding="utf-8") as f:
    dtypes_dict = json.load(f)

teste_oos_tratado = aplicar_dtypes(teste_oos, dtypes_dict)

#Imputar os Missings
#ler arquivo
with open("../model_artefacts/missing_plan.json", "r", encoding="utf-8") as f:
    prep = json.load(f)

teste_oos_tratado = apply_missing_prep(teste_oos_tratado, prep, copy=True)

In [0]:
teste_oos_tratado.shape

In [0]:
teste_oos_tratado.head()

In [0]:
teste_oos_tratado

In [0]:
import pandas as pd

VARIAVEIS_DE_OFERTA = [
    "offer_id","offer_type","discount_value","min_value","duration",
    "web","email","mobile","social"
]

def expandir_cenarios_ofertas(teste_oos_tratado: pd.DataFrame,
                              df_ofertas: pd.DataFrame,
                              variaveis_de_oferta=VARIAVEIS_DE_OFERTA,
                              add_id_cenario: bool = True) -> pd.DataFrame:
    """
    Para cada linha do teste, cria uma linha para cada oferta,
    substituindo SOMENTE as variáveis de oferta.
    Retorna um DF ~9x maior (cliente × oferta).
    """
    base = teste_oos_tratado.copy()
    ofertas = df_ofertas.copy()

    # checagens
    faltantes = [c for c in variaveis_de_oferta if c not in ofertas.columns]
    if faltantes:
        raise ValueError(f"df_ofertas está sem colunas: {faltantes}")

    # ---- cross join seguro ----
    try:
        # pandas >= 1.2
        cross = base.merge(ofertas, how="cross", suffixes=("", "_of"))
    except TypeError:
        # fallback: chave artificial __k SEM sufixar
        base["__k"] = 1
        ofertas["__k"] = 1
        cross = base.merge(ofertas, on="__k", how="inner", suffixes=("", "_of"))
        cross.drop(columns="__k", inplace=True)

    # ---- substituir apenas variáveis de oferta pelas versões _of ----
    # após o merge, as colunas que existem nos dois lados recebem sufixo "_of" no lado das ofertas
    for c in variaveis_de_oferta:
        c_of = f"{c}_of"
        if c_of in cross.columns:
            cross[c] = cross[c_of]
            cross.drop(columns=[c_of], inplace=True)
        # se não existe c_of, é porque não havia essa coluna no base e ela veio só do catálogo (ok)

    # ---- ordem de colunas: manter as do teste primeiro ----
    cols_orig = list(teste_oos_tratado.columns)
    cols_final = [c for c in cols_orig if c in cross.columns] + [c for c in cross.columns if c not in cols_orig]

    # ---- ID de cenário opcional ----
    if add_id_cenario and ("cliente_id" in cross.columns) and ("offer_id" in cross.columns):
        cross["ID_cenario"] = cross["cliente_id"].astype(str) + "_" + cross["offer_id"].astype(str)
        if "ID_cenario" not in cols_final:
            cols_final.append("ID_cenario")

    cross = cross[cols_final].reset_index(drop=True)
    return cross



In [0]:
expanded_oos = expandir_cenarios_ofertas(
    teste_oos_tratado=teste_oos_tratado,
    df_ofertas=offers,
    variaveis_de_oferta=VARIAVEIS_DE_OFERTA,
    add_id_cenario=True
)


In [0]:
expanded_oos.head(10)

In [0]:
expanded_oos.shape

In [0]:
#Agora podemos escorar o conjunto de testes
expanded_oos_ = aplicar_modelo_pkl(
    df_teste=expanded_oos[teste_oos_tratado.columns],              # df apenas com explicativas
    path='../model_artefacts/',
    score_col='score_modelo'
)

In [0]:
expanded_oos_.columns

In [0]:
melhor_offer_por_cliente = expanded_oos_[['cliente_id', 'offer_id','score_modelo',"time_since_test_start"]].sort_values(["cliente_id","offer_id","score_modelo"],ascending=False).drop_duplicates(["cliente_id","time_since_test_start"])

In [0]:
melhor_offer_por_cliente.shape

In [0]:
#Se tivéssemos escorado o cliente no dia 0, a melhor oferta seria a fafdcd668e3743c1bb461111dcafc2a4. 
#E assim por diante!
#Note que conforme o tempo passa, a probabilidade diminui, mas a melhor oferta ainda se mantém
melhor_offer_por_cliente.head(10)

In [0]:
#Nao houve cliente que mudou de "melhor oferta" ao longo do tempo, mas a probabilidade de conversao se mostrou variável
melhor_offer_por_cliente.groupby("cliente_id")["offer_id"].nunique().reset_index().head(8)


In [0]:
features = [
    "offer_type",
    "transacoes_distintas_dias",
    "n_instancias_anteriores_mesma_oferta",
    "limite_credito",
    "taxa_conversao_historica_bucket_minimo",
    "taxa_conversao_historica_tipo",
    "idade_missing",
    "taxa_conversao_historica_bucket_duracao",
    "discount_value",
    "media_tempo_entre_transacoes",
    "taxa_visualizacao_historica_oferta",
    "canais_recebidos_total",
    "safra_registro",
    "qtd_transacoes_14d",
    "genero",
    "min_value",
    "taxa_conversao",
    "dias_desde_ultima_transacao",
    "dias_desde_ultima_oferta_visualizada",
    "taxa_conversao_historica_bucket_desconto",
    "idade",
    "afinidade_canais_conv",
    "qtd_transacoes_7d",
    "qtd_ofertas_anteriores",
    "dias_desde_ultima_oferta_recebida",
    "email_ratio_historico",
    "social",
    "qtd_ofertas_recebidas_14d",
    "taxa_visualizacao_historica_tipo",
    "taxa_visualizacao",
    "taxa_conversao_historica_canal_social",
    "historico_conversao",
    "canais_preferidos",
    "duration",
    "qtd_conversoes_validas_14d",
    "qtd_ofertas_completas_validas",
    "mobile",
    "qtd_ofertas_visualizadas",
    "qtd_ofertas_visualizadas_14d",
    "recebeu_social",
    "web",
    "recebeu_web",
    "recebeu_mobile",
    "recebeu_email",
    "email"
]


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def simular_impacto(
    df,
    score_col="score_modelo",
    target_col="target_sucesso",
    cortes=(0.05, 0.10, 0.20, 0.30, 0.50, 1.00),
    valor_margem_por_conversao=10.0,   # R$ de margem por conversão
    custo_por_envio=0.02,              # R$ por cliente abordado
    custo_por_desconto=0.0,            # R$ por conversão (custo do cupom)
    nome_base="Teste"
):
    """
    Simula o impacto de abordar apenas o top-X% por score versus abordagem aleatória.

    Retorna:
      - DataFrame com: corte_pct, n_alvos, taxa_base, taxa_modelo, lift, conv_modelo,
        conv_random, conv_incrementais, receita_liquida_modelo, receita_liquida_random,
        uplift_liquido, ROI_modelo, ROI_random.
    """
    d = df[[score_col, target_col]].copy()
    d = d.dropna(subset=[score_col, target_col])
    d[score_col] = pd.to_numeric(d[score_col], errors="coerce").clip(0, 1)
    d = d.sort_values(score_col, ascending=False).reset_index(drop=True)

    N = len(d)
    tot_pos = int(d[target_col].sum())
    taxa_base = tot_pos / N if N else 0.0

    linhas = []
    for c in cortes:
        n_sel = int(np.floor(N * c))
        if n_sel <= 0:
            continue

        sel = d.iloc[:n_sel]
        conv_modelo = int(sel[target_col].sum())
        taxa_modelo = conv_modelo / n_sel

        # Cenário "random": mesma quantidade n_sel, taxa igual à média da base
        conv_random_esperadas = taxa_base * n_sel

        # Custos e receitas
        custo_envio = custo_por_envio * n_sel
        custo_desc_modelo = custo_por_desconto * conv_modelo
        receita_bruta_modelo = valor_margem_por_conversao * conv_modelo
        receita_liquida_modelo = receita_bruta_modelo - custo_envio - custo_desc_modelo

        custo_desc_random = custo_por_desconto * conv_random_esperadas
        receita_bruta_random = valor_margem_por_conversao * conv_random_esperadas
        receita_liquida_random = receita_bruta_random - custo_envio - custo_desc_random

        # Uplifts
        conv_incrementais = conv_modelo - conv_random_esperadas
        uplift_liquido = receita_liquida_modelo - receita_liquida_random

        inv_modelo = custo_envio + custo_desc_modelo
        inv_random = custo_envio + custo_desc_random
        ROI_modelo = (receita_liquida_modelo / inv_modelo) if inv_modelo > 0 else np.nan
        ROI_random = (receita_liquida_random / inv_random) if inv_random > 0 else np.nan

        linhas.append({
            "Base": nome_base,
            "corte_pct": c,
            "n_alvos": n_sel,
            "taxa_base": taxa_base,
            "taxa_modelo": taxa_modelo,
            "lift": (taxa_modelo / taxa_base) if taxa_base > 0 else np.nan,
            "conv_modelo": conv_modelo,
            "conv_random": conv_random_esperadas,
            "conv_incrementais": conv_incrementais,
            "custo_envio": custo_envio,
            "custo_descontos_modelo": custo_desc_modelo,
            "receita_liquida_modelo": receita_liquida_modelo,
            "receita_liquida_random": receita_liquida_random,
            "uplift_liquido": uplift_liquido,
            "ROI_modelo": ROI_modelo,
            "ROI_random": ROI_random
        })

    out = pd.DataFrame(linhas)
    # deixar números mais amigáveis
    num_cols = ["taxa_base","taxa_modelo","lift","conv_modelo","conv_random","conv_incrementais",
                "custo_envio","custo_descontos_modelo","receita_liquida_modelo",
                "receita_liquida_random","uplift_liquido","ROI_modelo","ROI_random"]
    for c in num_cols:
        if c in out.columns:
            out[c] = out[c].astype(float)

    return out


def simular_impacto_train_test(
    df_train, df_test,
    score_col="score_modelo",
    target_col="target_sucesso",
    cortes=(0.05, 0.10, 0.20, 0.30, 0.50, 1.00),
    valor_margem_por_conversao=10.0,
    custo_por_envio=0.02,
    custo_por_desconto=0.0,
    plot=True
):
    """
    Roda o simulador para Treino e Teste e (opcionalmente) plota Receita Líquida e Lift vs corte.
    """
    tab_train = simular_impacto(
        df_train, score_col, target_col, cortes,
        valor_margem_por_conversao, custo_por_envio, custo_por_desconto, nome_base="Treino"
    )
    tab_test = simular_impacto(
        df_test, score_col, target_col, cortes,
        valor_margem_por_conversao, custo_por_envio, custo_por_desconto, nome_base="Teste"
    )

    tab = pd.concat([tab_train, tab_test], ignore_index=True)

    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        for base, ax in zip(["Treino","Teste"], axes):
            sub = tab[tab["Base"]==base]
            ax.plot(sub["corte_pct"]*100, sub["receita_liquida_modelo"], marker="o", label="Modelo")
            ax.plot(sub["corte_pct"]*100, sub["receita_liquida_random"], marker="o", linestyle="--", label="Random")
            ax.set_title(f"{base} — Receita Líquida vs Corte")
            ax.set_xlabel("Top % dos clientes (score)")
            ax.set_ylabel("Receita líquida (R$)")
            ax.grid(alpha=0.3)
            ax.legend()

        plt.tight_layout()
        plt.show()

        # Lift
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        for base, ax in zip(["Treino","Teste"], axes):
            sub = tab[tab["Base"]==base]
            ax.plot(sub["corte_pct"]*100, sub["lift"], marker="o", color="#1f77b4")
            ax.axhline(1.0, color="gray", linestyle=":")
            ax.set_title(f"{base} — Lift vs Corte")
            ax.set_xlabel("Top % dos clientes (score)")
            ax.set_ylabel("Lift (taxa_modelo / taxa_base)")
            ax.grid(alpha=0.3)

        plt.tight_layout()
        plt.show()

    return tab


In [0]:
# Parâmetros de negócio — ajuste para sua realidade
VALOR_MARGEM = 12.0     # R$ de margem líquida por conversão (após custos variáveis)
CUSTO_ENVIO  = 0.03     # R$ por cliente abordado
CUSTO_CUPOM  = 1.50     # R$ por conversão (custo médio do desconto)

tabela_cenarios = simular_impacto_train_test(
    df_train=train_escorado, 
    df_test=teste_escorado,
    score_col="score_modelo",
    target_col="target_sucesso",
    cortes=(0.05, 0.10, 0.20, 0.30, 0.50, 0.60, 0.70, 0.80),
    valor_margem_por_conversao=VALOR_MARGEM,
    custo_por_envio=CUSTO_ENVIO,
    custo_por_desconto=CUSTO_CUPOM,
    plot=True
)

# Tabela com os números para o deck:
display(
    tabela_cenarios[[
        "Base","corte_pct","n_alvos",
        "taxa_base","taxa_modelo","lift",
        "conv_modelo","conv_random","conv_incrementais",
        "receita_liquida_modelo","receita_liquida_random","uplift_liquido","ROI_modelo"
    ]].assign(corte_pct=lambda x: (x["corte_pct"]*100).round(1))
)
